# 📥 Download e Preparação das Bases IBGE

Este notebook realiza o download, transformação e limpeza das bases IBGE utilizadas no projeto Polígonos de Uberlândia.

Fluxo:
1. Download dos arquivos
2. Conversão para formato `.parquet`
3. Remoção dos arquivos brutos


In [27]:
import os
import requests
import pandas as pd
import zipfile
import io
import openpyxl
import urllib.parse

In [28]:
bases_ibge = [
    'https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios_preliminares/malha_com_atributos/setores/gpkg/UF/MG/MG_Malha_Preliminar_2022.gpkg',
    'https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios_preliminares/agregados_por_setores_xlsx/BR/Agregados_preliminares_por_setores_censitarios_BR.xlsx',
    'https://ftp.ibge.gov.br/Cadastro_Nacional_de_Enderecos_para_Fins_Estatisticos/Censo_Demografico_2022/Arquivos_CNEFE/CSV/Municipio/31_MG/3170206_UBERL%c3%82NDIA.zip'
]

In [29]:
# 📌 Define o caminho da pasta onde vai salvar
pasta_IBGE = os.path.join('..', '..', 'bases', 'IBGE')
pasta_destino = os.path.join('..', '..', 'bases', 'tmp')
os.makedirs(pasta_destino, exist_ok=True)

## 🔽 1. Baixar arquivos IBGE


In [30]:
for link in bases_ibge:
    nome_arquivo = link.split('/')[-1]
    if '.gpkg' in nome_arquivo:
        caminho_completo = os.path.join(pasta_IBGE, nome_arquivo)
    else:
        caminho_completo = os.path.join(pasta_destino, nome_arquivo)

In [31]:
for link in bases_ibge:
    nome_arquivo = link.split('/')[-1]
    if '.gpkg' in nome_arquivo:
        caminho_completo = os.path.join(pasta_IBGE, nome_arquivo)
    else:
        caminho_completo = os.path.join(pasta_destino, nome_arquivo)   
    # Baixar
    resposta = requests.get(link)
    if resposta.status_code == 200:
        with open(caminho_completo, 'wb') as f:
            f.write(resposta.content)
        print(f"✅ Baixado: {nome_arquivo}")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo} - Status {resposta.status_code}")

✅ Baixado: MG_Malha_Preliminar_2022.gpkg
✅ Baixado: Agregados_preliminares_por_setores_censitarios_BR.xlsx
✅ Baixado: 3170206_UBERL%c3%82NDIA.zip


## 🛠 2. Ler arquivos e salvar em Parquet


In [32]:
arquivos = os.listdir(pasta_destino)
arquivos

['3170206_UBERL%c3%82NDIA.zip',
 'Agregados_preliminares_por_setores_censitarios_BR.xlsx']

In [33]:
import pandas as pd
import os
import zipfile

cidade = 3170206

for arq in arquivos:
    if '.gpkg' not in arq:
        caminho_completo = os.path.join(pasta_destino, arq)
        
        print(f'Lendo {arq}')
        
        if arq.endswith('.xlsx'):
            # Ler Excel
            df_tmp = pd.read_excel(caminho_completo)
            df_tmp = df_tmp[df_tmp['CD_MUN'] == cidade]
        
        elif arq.endswith('.zip'):
            # Ler CSV dentro do ZIP
            with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
                nome_csv = zip_ref.namelist()[0]
                with zip_ref.open(nome_csv) as f:
                    df_tmp = pd.read_csv(f, sep=';', encoding='utf-8')
        
        else:
            print(f'⚠️ Formato inesperado: {arq}')
            continue
        
        # 🔵 Trata colunas problemáticas: se tipo object e mistura texto com float
        for col in df_tmp.select_dtypes(include='object').columns:
            tipos = df_tmp[col].dropna().map(type).unique()
            if any(t in [float, int] for t in tipos) and any(t in [str, bytes] for t in tipos):
                print(f"⚙️ Corrigindo coluna '{col}' (mistura de tipos)")
                df_tmp[col] = df_tmp[col].astype('string')
        
        # 📌 Salvar cada DataFrame como Parquet
        nome_parquet = arq.replace('.xlsx', '.parquet').replace('.zip', '.parquet')
        nome_parquet = urllib.parse.unquote(nome_parquet)
        caminho_parquet = os.path.join(pasta_IBGE, nome_parquet)
        
        df_tmp.to_parquet(caminho_parquet, index=False, compression='snappy')
        print(f"✅ Salvo como Parquet: {nome_parquet}")


Lendo 3170206_UBERL%c3%82NDIA.zip


C:\Users\leand\AppData\Local\Temp\ipykernel_16460\2473383300.py:23: DtypeWarning: Columns (18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(f, sep=';', encoding='utf-8')


⚙️ Corrigindo coluna 'VAL_COMP_ELEM2' (mistura de tipos)
⚙️ Corrigindo coluna 'VAL_COMP_ELEM3' (mistura de tipos)
⚙️ Corrigindo coluna 'VAL_COMP_ELEM4' (mistura de tipos)
✅ Salvo como Parquet: 3170206_UBERLÂNDIA.parquet
Lendo Agregados_preliminares_por_setores_censitarios_BR.xlsx
✅ Salvo como Parquet: Agregados_preliminares_por_setores_censitarios_BR.parquet


## 🧹 3. Limpar arquivos brutos


In [34]:
# 📌 Deleta todos que não forem .parquet ou .gpkg
for arq in arquivos:
    if not arq.endswith('.parquet') and not arq.endswith('.gpkg'):
        caminho_arquivo = os.path.join(pasta_destino, arq)
        try:
            os.remove(caminho_arquivo)
            print(f"🗑️ Arquivo deletado: {arq}")
        except Exception as e:
            print(f"❌ Erro ao tentar deletar {arq}: {e}")

🗑️ Arquivo deletado: 3170206_UBERL%c3%82NDIA.zip
🗑️ Arquivo deletado: Agregados_preliminares_por_setores_censitarios_BR.xlsx
